In [ ]:
# !pip install tensorflow scikit-learn pandas matplotlib --quiet

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_fscore_support, RocCurveDisplay
import tensorflow as tf
from tensorflow.keras import layers, regularizers, callbacks, Model

# ----------------------------
# 1) Load data
# ----------------------------
CSV_PATH = r"C:\Users\HP\Downloads\creditcard.csv"  # <- change if needed

df = pd.read_csv(CSV_PATH)
# Features: V1..V28 are already PCA-like; scale 'Time' and 'Amount'
features = [c for c in df.columns if c != "Class"]
X = df[features].copy()
y = df["Class"].astype(int).values

# Keep a copy of original columns for reference
print("Shape:", X.shape, "| Frauds:", y.sum(), "| Normal:", (y==0).sum())

# ----------------------------
# 2) Train/Val/Test split (train ONLY on normal)
# ----------------------------
# First, carve out a test set stratified by class
X_trval, X_test, y_trval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# From the remaining, create a training set of normals only
X_trval_norm = X_trval[y_trval == 0]
X_train_norm, X_val_norm = train_test_split(
    X_trval_norm, test_size=0.2, random_state=42
)

# Create a small validation set with both normal and fraud to pick threshold robustly
X_val_mix = X_trval.copy()
y_val_mix = y_trval.copy()

# ----------------------------
# 3) Scale features (fit ONLY on normal train to avoid leakage)
# ----------------------------
scaler = StandardScaler()
X_train_norm_s = scaler.fit_transform(X_train_norm)
X_val_norm_s   = scaler.transform(X_val_norm)
X_val_mix_s    = scaler.transform(X_val_mix)
X_test_s       = scaler.transform(X_test)

input_dim = X_train_norm_s.shape[1]

# ----------------------------
# 4) Build Autoencoder
#    (small, L1 activity regularizer helps sparsity a bit)
# ----------------------------
inp = layers.Input(shape=(input_dim,))
x   = layers.Dense(32, activation="relu")(inp)
x   = layers.Dense(16, activation="relu",
                   activity_regularizer=regularizers.l1(1e-5))(x)
enc = layers.Dense(8,  activation="relu", name="encoded")(x)

x   = layers.Dense(16, activation="relu")(enc)
x   = layers.Dense(32, activation="relu")(x)
out = layers.Dense(input_dim, activation="linear")(x)  # reconstruct raw scaled features

autoencoder = Model(inp, out, name="ae")
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.summary()

# ----------------------------
# 5) Train (only normal data)
# ----------------------------
es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

hist = autoencoder.fit(
    X_train_norm_s, X_train_norm_s,
    validation_data=(X_val_norm_s, X_val_norm_s),
    epochs=100,
    batch_size=512,
    shuffle=True,
    callbacks=[es],
    verbose=1
)

# A) Percentile on normal val errors
val_norm_recon = autoencoder.predict(X_val_norm_s, verbose=0)
val_norm_err = np.mean((val_norm_recon - X_val_norm_s)**2, axis=1)
thr_percentile = np.percentile(val_norm_err, 99.5)  # 99.5th percentile (tune)

val_mix_recon = autoencoder.predict(X_val_mix_s, verbose=0)
val_mix_err = np.mean((val_mix_recon - X_val_mix_s)**2, axis=1)

# Grid thresholds from quantiles of val_mix_err
qs = np.linspace(80, 99.9, 60)
best_f1, best_thr = -1, None
for q in qs:
    thr = np.percentile(val_mix_err, q)
    y_pred = (val_mix_err > thr).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_val_mix, y_pred, average="binary", zero_division=0)
    if f1 > best_f1:
        best_f1, best_thr = f1, thr

print(f"\n[Thresholds] percentile(99.5)={thr_percentile:.6f} | best_F1={best_f1:.4f} at thr={best_thr:.6f}")

# Use the F1-optimized threshold by default (fallback to percentile if None)
threshold = best_thr if best_thr is not None else thr_percentile
print(f"Chosen threshold = {threshold:.6f}")

# ----------------------------
# 7) Evaluate on TEST set
# ----------------------------
test_recon = autoencoder.predict(X_test_s, verbose=0)
test_err = np.mean((test_recon - X_test_s)**2, axis=1)
y_pred = (test_err > threshold).astype(int)  # 1 = anomaly (fraud)

print("\nConfusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred, digits=4))

auc = roc_auc_score(y_test, test_err)  # AUC using scores (higher=more anomalous)
print(f"AUC (reconstruction error as score): {auc:.4f}")

# ----------------------------
# 8) Quick visuals
# ----------------------------
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(test_err[y_test==0], bins=50, alpha=0.6, label="Normal")
plt.hist(test_err[y_test==1], bins=50, alpha=0.6, label="Fraud")
plt.axvline(threshold, color='r', linestyle='--', label='Threshold')
plt.title("Reconstruction Error (Test)")
plt.xlabel("MSE")
plt.ylabel("Count")
plt.legend()

plt.subplot(1,2,2)
RocCurveDisplay.from_predictions(y_test, test_err, pos_label=1)  # scores, not binary
plt.title("ROC (using reconstruction error)")
plt.show()


Shape: (284807, 30) | Frauds: 492 | Normal: 284315


Model: "ae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,334 (13.02 KB)

 Trainable params: 3,334 (13.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.8463 - val_loss: 0.6997
Epoch 2/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6200 - val_loss: 0.5615
Epoch 3/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5281 - val_loss: 0.4932
Epoch 4/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4766 - val_loss: 0.4577
Epoch 5/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4488 - val_loss: 0.4333
Epoch 6/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4266 - val_loss: 0.4147
Epoch 7/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4095 - val_loss: 0.4023
Epoch 8/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3941 - val_loss: 0.3867
Epoch 9/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3831 - val_loss: 0.3770
Epoch 10/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3759 - val_loss: 0.3708
Epoch 11/100
356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3670 - val_loss: 0.3630
Epoch 12/100
356/356 ━━━━━━━━━━━━━━━━━━━━

NameError: name 'val_mix_err' is not defined